# Create a SageMaker MLOps Project for Pipelines
Automatically run pipelines when code changes using Amazon SageMaker Projects

Note:  This requires that you have enabled products within SageMaker Studio

![](../img/enable-service-catalog-portfolio-for-studio.png)

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
sc = boto3.Session().client(service_name="servicecatalog", region_name=region)
sts = boto3.Session().client(service_name="sts", region_name=region)
iam = boto3.Session().client(service_name="iam", region_name=region)
codepipeline = boto3.Session().client("codepipeline", region_name=region)

In [ ]:
search_response = sc.search_products(
    Filters={"FullTextSearch": ["MLOps template for model building, training, and deployment"]}
)

sagemaker_pipeline_product_id = search_response["ProductViewSummaries"][0]["ProductId"]
print(sagemaker_pipeline_product_id)

In [ ]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response["ProvisioningArtifacts"][-1]["Id"]

In [ ]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

# Create a SageMaker Project


In [ ]:
import time

timestamp = int(time.time())

In [ ]:
sagemaker_project_name = "dsoaws-{}".format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription="dsoaws-{}".format(timestamp),
    ServiceCatalogProvisioningDetails={
        "ProductId": sagemaker_pipeline_product_id,
        "ProvisioningArtifactId": sagemaker_pipeline_product_provisioning_artifact_id,
    },
)

sagemaker_project_id = create_response["ProjectId"]
sagemaker_project_arn = create_response["ProjectArn"]

print("Project ID {}".format(sagemaker_project_id))
print("Project ARN {}".format(sagemaker_project_arn))

In [ ]:
sagemaker_project_name_and_id = "{}-{}".format(sagemaker_project_name, sagemaker_project_id)

print("Combined Project ID and ARN combined: {}".format(sagemaker_project_name_and_id))

# _Wait for the Project to be Created_

In [ ]:
%%time

import time

try:
    describe_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = describe_project_response["ProjectStatus"]
    print("Creating Project...")

    while project_status in ["Pending", "CreateInProgress"]:
        print("Please wait...")
        time.sleep(30)
        describe_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = describe_project_response["ProjectStatus"]
        print("Project status: {}".format(project_status))

    if project_status == "CreateCompleted":
        print("Project {}".format(project_status))

    else:
        print("Project status: {}".format(project_status))
        raise Exception("Project not created.")

except Exception as e:
    print(e)

print(describe_project_response)

# _Wait for Project to be Created ^^ Above ^^_

# Attach IAM Policies for FeatureStore 
This is the role used by Code Build when it starts the pipeline.

In [ ]:
sc_role_name = "AmazonSageMakerServiceCatalogProductsUseRole"

In [ ]:
account_id = sts.get_caller_identity()["Account"]
print(account_id)

In [ ]:
response = iam.attach_role_policy(RoleName=sc_role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess")

print(response)

In [ ]:
response = iam.attach_role_policy(
    RoleName=sc_role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess"
)

print(response)

In [ ]:
response = iam.attach_role_policy(RoleName=sc_role_name, PolicyArn="arn:aws:iam::aws:policy/IAMFullAccess")

print(response)

# Stop the `Abalone` Sample Pipeline that Ships with SageMaker Pipelines
The sample "abalone" pipeline starts automatically when we create the project.  We want to stop this pipeline to release these resources and use them for our own pipeline.

In [ ]:
sample_abalone_pipeline_execution_arn = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)["PipelineExecutionSummaries"][0]["PipelineExecutionArn"]

print(sample_abalone_pipeline_execution_arn)

In [ ]:
sm.stop_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)

In [ ]:
%%time

try:
    describe_pipeline_execution_response = sm.describe_pipeline_execution(
        PipelineExecutionArn=sample_abalone_pipeline_execution_arn
    )
    pipeline_execution_status = describe_pipeline_execution_response["PipelineExecutionStatus"]

    while pipeline_execution_status not in ["Stopped", "Failed"]:
        print("Please wait...")
        time.sleep(30)
        describe_pipeline_execution_response = sm.describe_pipeline_execution(
            PipelineExecutionArn=sample_abalone_pipeline_execution_arn
        )
        pipeline_execution_status = describe_pipeline_execution_response["PipelineExecutionStatus"]
        print("Pipeline execution status: {}".format(pipeline_execution_status))

    if pipeline_execution_status in ["Stopped", "Failed"]:
        print("Pipeline execution status {}".format(pipeline_execution_status))
    else:
        print("Pipeline execution status: {}".format(pipeline_execution_status))
        raise Exception("Pipeline execution not deleted.")

except Exception as e:
    print(e)

print(describe_pipeline_execution_response)

In [ ]:
sm.delete_pipeline(PipelineName=sagemaker_project_name_and_id)

# Clone the MLOps Repositories in AWS CodeCommit

In [ ]:
import os

sm_studio_root_path = "/root/"
sm_notebooks_root_path = "/home/ec2-user/SageMaker/"

root_path = sm_notebooks_root_path if os.path.isdir(sm_notebooks_root_path) else sm_studio_root_path

print(root_path)

In [ ]:
print(region)

In [ ]:
code_commit_repo1 = "https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild".format(
    region, sagemaker_project_name_and_id
)
print(code_commit_repo1)

In [ ]:
sagemaker_mlops_build_code = "{}{}/sagemaker-{}-modelbuild".format(
    root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id
)
print(sagemaker_mlops_build_code)

In [ ]:
code_commit_repo2 = "https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy".format(
    region, sagemaker_project_name_and_id
)
print(code_commit_repo2)

In [ ]:
sagemaker_mlops_deploy_code = "{}{}/sagemaker-{}-modeldeploy".format(
    root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id
)
print(sagemaker_mlops_deploy_code)

In [ ]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

In [ ]:
!git clone $code_commit_repo1 $sagemaker_mlops_build_code

In [ ]:
!git clone $code_commit_repo2 $sagemaker_mlops_deploy_code

# Remove Sample `Abalone` Example Code

In [ ]:
!rm -rf $sagemaker_mlops_build_code/pipelines/abalone

# Copy Workshop Code Into Local Project Folders

In [ ]:
workshop_project_build_code = "{}workshop/10_pipeline/sagemaker_mlops/sagemaker-project-modelbuild".format(root_path)
print(workshop_project_build_code)

In [ ]:
workshop_project_deploy_code = "{}workshop/10_pipeline/sagemaker_mlops/sagemaker-project-modeldeploy".format(root_path)
print(workshop_project_deploy_code)

In [ ]:
!cp -R $workshop_project_build_code/* $sagemaker_mlops_build_code/

In [ ]:
!cp -R $workshop_project_deploy_code/* $sagemaker_mlops_deploy_code/

# Commit New Code 

In [ ]:
print(sagemaker_mlops_build_code)

In [ ]:
!cd $sagemaker_mlops_build_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

In [ ]:
!cd $sagemaker_mlops_deploy_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

# Store the Variables

In [ ]:
%store sagemaker_mlops_build_code
%store sagemaker_mlops_deploy_code
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

In [ ]:
!ls -al $sagemaker_mlops_build_code/pipelines/dsoaws/

In [ ]:
!pygmentize $sagemaker_mlops_build_code/pipelines/dsoaws/pipeline.py

# Wait for Pipeline Execution to Start
Now that we have committed code, our pipeline will start.  Let's wait for the pipeline to start.


In [ ]:
%%time

import time
from pprint import pprint

while True:
    try:
        print("Listing executions for our pipeline...")
        list_executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)[
            "PipelineExecutionSummaries"
        ]
        break
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(list_executions_response)

In [ ]:
build_pipeline_name = "sagemaker-{}-modelbuild".format(sagemaker_project_name_and_id)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Check <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">ModelBuild</a> Pipeline</b>'.format(
            build_pipeline_name, region
        )
    )
)

# Wait For Pipeline Execution To Complete

In [ ]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)[
    "PipelineExecutionSummaries"
]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)[
            "PipelineExecutionSummaries"
        ]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    #        print('Executions for our pipeline...')
    #        print(pipeline_execution_status)
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

# _Wait for the Pipeline Execution ^^ Above ^^ to Complete_

# List Pipeline Execution Steps

In [ ]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

In [ ]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

In [ ]:
pd.set_option("max_colwidth", 1000)

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Approve the Registered Model for Staging
The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

Notes:  
* You can do this within SageMaker Studio, as well.  However, we are approving programmatically here in this example.
* This approval is only for Staging.  For Production, you must go through the CodePipeline (deep link is below.)


In [ ]:
%%time

import time
from pprint import pprint

while True:
    try:
        print("Executions for our pipeline...")
        list_model_packages_response = sm.list_model_packages(ModelPackageGroupName=sagemaker_project_name_and_id)
        break
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(list_model_packages_response)

In [ ]:
time.sleep(30)

model_package_arn = list_model_packages_response["ModelPackageSummaryList"][0]["ModelPackageArn"]
print(model_package_arn)

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

In [ ]:
time.sleep(30)

model_name = sm.list_models()["Models"][0]["ModelName"]
print(model_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/models/{}">Model</a></b>'.format(
            region, model_name
        )
    )
)

In [ ]:
deploy_pipeline_name = "sagemaker-{}-modeldeploy".format(sagemaker_project_name_and_id)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Check <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">ModelDeploy</a> Pipeline</b>'.format(
            deploy_pipeline_name, region
        )
    )
)

In [ ]:
staging_endpoint_name = "{}-staging".format(sagemaker_project_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker Staging REST Endpoint</a></b>'.format(
            region, staging_endpoint_name
        )
    )
)

# _Wait Until the Staging Endpoint is Deployed_


In [ ]:
%%time

while True:
    try:
        waiter = sm.get_waiter("endpoint_in_service")
        print("Waiting for staging endpoint to be in `InService`...")
        waiter.wait(EndpointName=staging_endpoint_name)
        break
    except:
        print("Waiting for staging endpoint to be in `Creating`...")
        time.sleep(30)

print("Staging endpoint deployed.")

# _Wait Until the ^^ Staging Endpoint ^^ is Deployed_

# List Artifact Lineage

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Run A Sample Prediction in Staging

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=staging_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

In [ ]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

# Deploy to Production

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-modeldeploy/view?region={}"> Deploy to Production </a> Pipeline</b> '.format(
            sagemaker_project_name_and_id, region
        )
    )
)

In [ ]:
stage_name = "DeployStaging"
action_name = "ApproveDeployment"

In [ ]:
time.sleep(60)

stage_states = codepipeline.get_pipeline_state(name=deploy_pipeline_name)["stageStates"]

for stage_state in stage_states:

    if stage_state["stageName"] == stage_name:
        for action_state in stage_state["actionStates"]:
            if action_state["actionName"] == action_name:
                token = action_state["latestExecution"]["token"]

print(token)

In [ ]:
response = codepipeline.put_approval_result(
    pipelineName=deploy_pipeline_name,
    stageName=stage_name,
    actionName=action_name,
    result={"summary": "Approve from Staging to Production", "status": "Approved"},
    token=token,
)

# Get the Production Endpoint Name

In [ ]:
time.sleep(30)

production_endpoint_name = "{}-prod".format(sagemaker_project_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker Production REST Endpoint</a></b>'.format(
            region, production_endpoint_name
        )
    )
)

# _Wait Until the Production Endpoint is Deployed_

In [ ]:
%%time

while True:
    try:
        waiter = sm.get_waiter("endpoint_in_service")
        print("Waiting for production endpoint to be in `InService`...")
        waiter.wait(EndpointName=production_endpoint_name)
        break
    except:
        print("Waiting for production endpoint to be in `Creating`...")
        time.sleep(30)

print("Production endpoint deployed.")

# _Wait Until the ^^ Production Endpoint ^^ is Deployed_

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=production_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

In [ ]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

# List Artifact Lineage

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}